In [1]:
import re
import numpy as np
from collections import defaultdict
from queue import SimpleQueue
from tqdm import tqdm
import math
import itertools
from functools import cache

with open("data.txt") as file:
    data = file.read().splitlines()

In [2]:
input = {
    re.search("[a-z]+(?= )", a).group(): [
        re.search("[%&]|broadcaster", a).group(),
        re.findall(r"[a-z]+", a.split("-> ")[-1]),
    ]
    for a in data
}

In [82]:
def press_button(n, check = "", prev = ""):

    flipflop = defaultdict(bool)
    conjunctions = [key for key, val in input.items() if val[0] == "&"]
    conjunctions = {
        c: {key: 0 for key, val in input.items() if c in val[1]} for c in conjunctions
    }

    high, low = 0,0

    for button_press in range(n):

        que = SimpleQueue()
        que.put(("broadcaster", 0, ""))

        while que.qsize() > 0:
            key, signal, previous = que.get()

            if signal == 0:
                low += 1
            else:
                high += 1

            current = input.get(key, [None])

            if current[0] == "%" and signal == 0:
                flipflop[key] = not flipflop[key]
                signal_out = int(flipflop[key])

                for n in current[1]:
                    que.put((n, signal_out, key))

            if current[0] == "&":
                conjunction_dict = conjunctions[key]
                conjunction_dict[previous] = signal

                signal_out = 1 - int(
                    len(conjunction_dict)
                    == len([a for a in conjunction_dict.values() if a == 1])
                )

                for n in current[1]:
                    que.put((n, signal_out, key))

            if current[0] == "broadcaster":

                for n in current[1]:
                    que.put((n, 0, key))
            
            if key == check and signal == 1 and previous in prev:
                return button_press

    return high, low

np.prod(press_button(1000, check= "tg", prev = ""))

819397964

In [83]:
before_rx = [k for k,v in input.items() if 'rx' in v[1]][0]
before_before_rx = [k for k,v in input.items() if before_rx in v[1]]
np.prod([press_button(10000, check= before_rx, prev = a) + 1 for a in before_before_rx])

252667369442479